In [1]:
import yfinance as yf
import boto
from os import environ as env

In [6]:
#!/usr/bin/env python
# coding: utf-8


def calc_stock(high, current):
    ratio = (current - high) / high
    return ratio


def convert_tuple(tup):
    string_tup = "{} : {}".format(tup[0], str(tup[1]))
    return string_tup


def create_message(pairs):
    message = "\nORDERED RATIOS:\n\n"
    for pair in pairs:
        message += convert_tuple(pair) + "\n"
    return message


def publish_message_sns(message):
    sns_arn = env.get('SNS_ARN').strip()
    sns_client = boto3.client('sns')
    try:
        response = sns_client.publish(
            TopicArn=sns_arn,
            Message=message
        )

        print(response)

    except Exception as e:
        print("ERROR PUBLISHING MESSAGE TO SNS: {}".format(e))


def read_tickers(file='deployment/stock_tickers.txt', period='5y'):
    pairs = dict()
    out_string = "\nINDIVIDUAL STATS:\n"

    with open(file, 'r') as f:
        while True:
            temp_string = ""
            try:
                # Get next line from file
                ticker = (f.readline()).strip()

                if ticker == "":
                    break

                stock = yf.Ticker(ticker)
                data = stock.history(env.get('PERIOD', period))

                close = data.Close[-1]
                temp_string += "{} Close: {}\n".format(ticker, str(close))

                high = max(data.Close)
                temp_string += "{} {}-High: {}\n".format(ticker, env.get('PERIOD', period), str(high))

                delta = calc_stock(high, close)

                temp_string += "{} Delta: {}\n".format(ticker, str(delta))

                out_string += "\n" + temp_string

                pairs[ticker] = delta

                if not ticker:
                    break

            except Exception as e:
                print(e)
                if not ticker:
                    break

    return out_string, sorted(pairs.items(), key=lambda x: x[1])


def handler():
    """
    This function drives the AWS lambda. Requires 1 env var to work correctly: SNS_TOPIC which represents the topic arn to which
    you want to publish. 
    """

    out_string, pairs = read_tickers()
    message = create_message(pairs)
    message += out_string
    print(message)
#     publish_message_sns(message)



In [7]:
handler()


ORDERED RATIOS:

UCO : -0.9865938430983119
AMC : -0.8849139568970197
SAVE : -0.8337185413456599
DAL : -0.6203629356030191
WFC : -0.5457827371184771
LUV : -0.5180760279286268
COF : -0.49346497414198404
C : -0.45913727417967315
AGN : -0.41559332605644783
CVS : -0.3833217924621624
SQ : -0.3832946167053833
AXP : -0.3830505985165601
DIS : -0.321613374021494
CRSP : -0.273542600896861
SPOT : -0.2636030160994498
CSCO : -0.24920578891634307
IBM : -0.24512569744843582
WORK : -0.24262040393578452
DOC : -0.23811856932876038
V : -0.23013454596596505
SBUX : -0.2297780959198283
ABBV : -0.22446906740535552
CMCSA : -0.2114854842127569
FB : -0.20154101151278941
T : -0.1938589643507676
DDOG : -0.18036392721455702
GOOG : -0.17035547491632233
HD : -0.16028201157388533
CRM : -0.1582540339263551
NKE : -0.15715792501678014
NOW : -0.15447836296544795
AAPL : -0.15363691931540338
INTU : -0.13428430890309884
INTC : -0.13136650521062668
SAP : -0.12569553431302613
LMT : -0.12330992198760045
TMUS : -0.1049855707035